In [104]:
# TO DO:
#readme:
    #challenge - # ensure places aren't repeated.  get them dupes out

# do iterative 50 at a time thingy

# markdown

#extra fun:
    # Identify if we can call config from separate spot in repo

    # Extract subnational unit ie state where appropriate
        # list as challenge?

    # Print summary rollup of missed cities, and save log to txt file
        # only print summary records to readout here, reference full log file for further details.
        # https://www.pythontutorial.net/python-basics/python-create-text-file/
        # Enable verbose logging heh
    
    # Explore other fun variables to track in provided weather data.

In [106]:
# Namespace
# Modules
import numpy as np
import pandas as pd
import requests
import string
# import os
from citipy import citipy
from pprint import pprint

# api keys, etc.  Is there a better way to import this than duplicating this in a second spot in the repo?
from config import weather_api_key

In [107]:
num_cities = 2000

# generate 2k random lat/long values
lats = np.random.uniform(low=-90.0, high=90.0, size=num_cities)
longs = np.random.uniform(low=-180.0, high=180.0, size=num_cities)
coords_zip = zip(lats,longs)
coords = list(coords_zip)

In [114]:
#instantiate base URL
base_url = "http://api.openweathermap.org/data/2.5/weather"

# instantiate list of results. will be comprised of dictionaries.
weather_data = list()

# instantiate list of full placenames (city plus country code) to eliminate dupes
full_placenames = list()

# Statistics for logging
dupe_count = 0
city_not_found_count = 0

for i in range(50):
    # use citypy to get nearest cities
    city_obj = citipy.nearest_city(latitude=coords[i][0], longitude=coords[i][1])

    # Lower case strings for formal place names bother me.  So we're fixing that.
    city_name = string.capwords(city_obj.city_name)
    country_code = city_obj.country_code.upper()
    
    full_placename = f"{city_name},{country_code}"

    if full_placename not in full_placenames:

        full_placenames.append(full_placename)
        
        params={'appid':weather_api_key,
                'q':full_placename,
                'units':'imperial'
               }

        weather_response = requests.get(url=base_url,params=params)

        try:
            weather_json = weather_response.json()

            weather_data.append({
                "City" : city_name,
                "Country" : country_code,
                "Lat" : weather_json['coord']['lat'],
                "Long" : weather_json['coord']['lon'],
                "Max Temp" : weather_json['main']['temp_max'],
                "Pct Humidity" : weather_json['main']['humidity'],
                "Pct Cloudiness" : weather_json['clouds']['all'],
                "Wind Speed" : weather_json['wind']['speed'],
                "Description" : weather_json['weather'][0]['description']
            })

            print(f"Successfully processed {full_placename}.")
        except KeyError:
            print(f"API call failed to retrieve data for {full_placename}.  Error code: {weather_response.status_code}.")
            city_not_found_count += 1
        
    else:
        print(f"{full_placename} already exists in data set.  Skipping.")
        dupe_count += 1

print("------------------------------------------")
print("Processing complete.  BUT I NEED A TIMESTAMP.")
print(f"Number of coordinates generated: {num_cities}")
print(f"Number of duplicate city records identified: {dupe_count}")
print(f"Total unique cities identified: {len(full_placenames)}")
print("------------------------------------------")
print(f"Number of cities with unidentifiable weather reports: {city_not_found_count}") # REWORD ME
print(f"Successfully processed {len(weather_data)} total records.")

Successfully processed Rafai,CF.
Successfully processed Saskylakh,RU.
Successfully processed Nanortalik,GL.
Successfully processed Koroni,GR.
Successfully processed Mayo,CA.
Successfully processed Rocha,UY.
Successfully processed Saint-philippe,RE.
Successfully processed Hilo,US.
Successfully processed Vestmannaeyjar,IS.
Successfully processed Hobart,AU.
Successfully processed Matara,LK.
Successfully processed Mount Gambier,AU.
API call failed to retrieve data for Taolanaro,MG.  Error code: 404.
Successfully processed Ushuaia,AR.
Successfully processed Ponta Do Sol,CV.
Successfully processed Butaritari,KI.
Successfully processed Rikitea,PF.
Successfully processed Canakkale,TR.
Saint-philippe,RE already exists in data set.  Skipping.
Successfully processed Cape Town,ZA.
Successfully processed Mazatlan,MX.
Successfully processed Tiksi,RU.
Successfully processed Quatre Cocos,MU.
Taolanaro,MG already exists in data set.  Skipping.
Successfully processed Victoria,SC.
Successfully processed 

In [110]:
weather_data_df = pd.DataFrame(weather_data)

weather_data_df.to_csv("WeatherPy_Database.csv")

weather_data_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Rafai,CF,4.9500,23.9167,81.93,32,72,2.62,broken clouds
1,Saskylakh,RU,71.9167,114.0833,-35.14,100,24,2.95,few clouds
2,Nanortalik,GL,60.1432,-45.2371,22.62,70,7,5.37,clear sky
3,Koroni,GR,36.8000,21.9500,51.51,67,72,10.51,broken clouds
4,Mayo,CA,63.5927,-135.8998,-5.80,84,90,0.00,overcast clouds
5,Rocha,UY,-34.4833,-54.3333,96.76,19,12,10.18,few clouds
6,Saint-philippe,RE,-21.3585,55.7679,78.39,74,93,12.86,overcast clouds
7,Hilo,US,19.7297,-155.0900,69.80,46,90,5.75,moderate rain
8,Vestmannaeyjar,IS,63.4427,-20.2734,32.99,59,20,5.75,few clouds
9,Hobart,AU,-42.8794,147.3294,56.12,82,91,4.61,overcast clouds
